In [6]:
from croqr.data.deribit.config import get_deribit_sample_data_url
from croqr.data.deribit.data_utils import url_csv_zip_file_reader
from croqr.analytics.options.option_utils import euro_vanilla
from croqr.analytics.options.option_utils import find_vol
from croqr.common.dt_utils import DataTimeUtil
from croqr.common.config import SECONDS_PER_DAY
from croqr.analytics.options.option_utils import euro_vanilla, Put

## Load Deribit sample data

In [3]:
url = get_deribit_sample_data_url('OPTIONS', 'options_chain')
df = url_csv_zip_file_reader(url)

loading cached Deribit_OPTIONS_options_chain_2020-07-01_2020-07-31.csv.gz
file size > 500MB, load top 5000 rows


In [4]:
df.head(5)

,symbol,timestamp,local_timestamp,type,strike_price,expiration,open_interest,last_price,bid_price,bid_amount,...,ask_iv,mark_price,mark_iv,underlying_index,underlying_price,delta,gamma,vega,theta,rho
exchange,,,,,,,,,,,,,,,,,,,,,
deribit,BTC-3JUL20-9500-C,1593561597178000,1593561600161993,call,9500,1593763200000000,1186.0,0.0035,0.0030,0.7,...,52.12,0.003461,49.50,SYN.BTC-3JUL20,9136.98,0.16736,0.00069,1.83013,-19.41256,0.09573
deribit,BTC-3JUL20-9250-P,1593561598001000,1593561600161994,put,9250,1593763200000000,785.0,0.0205,0.0195,0.3,...,48.45,0.022261,47.63,SYN.BTC-3JUL20,9137.12,-0.61919,0.00109,2.78342,-28.40972,-0.37467
deribit,BTC-3JUL20-8250-P,1593561592616000,1593561600161995,put,8250,1593763200000000,468.8,0.0015,0.0010,8.2,...,80.88,0.001244,77.79,SYN.BTC-3JUL20,9136.84,-0.04719,0.00017,0.71940,-11.36439,-0.02829
deribit,BTC-3JUL20-7750-P,1593561597884000,1593561600161997,put,7750,1593763200000000,306.3,0.0005,NaN,NaN,...,110.75,0.000283,91.41,SYN.BTC-3JUL20,9137.12,-0.01103,0.00004,0.21207,-2.58385,-0.00661
deribit,BTC-3JUL20-6750-C,1593561594862000,1593561600161998,call,6750,1593763200000000,0.0,NaN,0.0855,11.5,...,500.00,0.261225,91.78,SYN.BTC-3JUL20,9136.71,0.99998,0.00000,0.00050,-0.00991,0.43150


In [5]:
all_symbols = list(df['symbol'].unique())

In [6]:
dates = sorted(list(set([x.split('-')[1] for x in all_symbols])))

In [7]:
df['timestamp_s'], df['microsec'] = zip(*df['timestamp'].apply(DataTimeUtil.epoch_time_converter, 1))
df['exp_timestamp_s'], df['exp_microsec'] = zip(*df['expiration'].apply(DataTimeUtil.epoch_time_converter, 1))

In [8]:
df['underlying_index'].unique()

array(['SYN.BTC-3JUL20', 'SYN.BTC-31JUL20', 'SYN.BTC-2JUL20',
       'SYN.BTC-28AUG20', 'BTC-26MAR21', 'BTC-25SEP20', 'BTC-25DEC20',
       'SYN.BTC-1JUL20', 'SYN.BTC-10JUL20', 'SYN.ETH-3JUL20',
       'SYN.ETH-31JUL20', 'SYN.ETH-2JUL20', 'SYN.ETH-28AUG20',
       'ETH-26MAR21', 'ETH-25SEP20', 'ETH-25DEC20', 'SYN.ETH-1JUL20',
       'SYN.ETH-10JUL20'], dtype=object)

In [9]:
opt1 = df.iloc[1,:]
opt1

symbol                BTC-3JUL20-9250-P
timestamp              1593561598001000
local_timestamp        1593561600161994
type                                put
strike_price                       9250
expiration             1593763200000000
open_interest                     785.0
last_price                       0.0205
bid_price                        0.0195
bid_amount                          0.3
bid_iv                            38.49
ask_price                        0.0225
ask_amount                          0.3
ask_iv                            48.45
mark_price                     0.022261
mark_iv                           47.63
underlying_index         SYN.BTC-3JUL20
underlying_price                9137.12
delta                          -0.61919
gamma                           0.00109
vega                            2.78342
theta                         -28.40972
rho                            -0.37467
timestamp_s         2020-07-01 00:59:58
microsec                           1000


### Verify BS formula

In [10]:
T = (opt1['expiration'] - opt1['timestamp'])/(1e6*SECONDS_PER_DAY*365)
T

0.006392757451801116

In [11]:
target_value = opt1['bid_price']
S = opt1['underlying_price']
K = opt1['strike_price']
sigma = opt1['ask_iv']/100
r = 0 

In [12]:
print('S={},'.format(S), 'K={},'.format(K), 'T={},'.format(T), 'r={},'.format(r), 'sgima={},'.format(sigma))

S=9137.12, K=9250, T=0.006392757451801116, r=0, sgima=0.48450000000000004,


In [13]:
put_ask_iv = euro_vanilla(S, K, T, r, sigma, option='put')

In [14]:
put_ask_iv, opt1['ask_price']*opt1['underlying_price']

(205.58690614397256, 205.58520000000001)

### chceck greeks using opstrat

In [15]:
Put.Price?

Signature: Put.Price(S, K, r, sigma, T)
Docstring: <no docstring>
File:      c:\users\wang\documents\github\croqr\croqr\analytics\options\option_utils.py
Type:      function


In [16]:
Put.Price(S, K, r, sigma, T)

205.58690614397256

In [17]:
Put.Delta(S, K, r, sigma, T)

-0.6169913099694828

In [18]:
Put.Gamma(S, K, r, sigma, T)

0.0010782824572031426

In [22]:
Put.Vega(S, K, r, sigma, T)/100

2.788259595507463

In [27]:
Put.Theta(S, K, r, sigma, T)/365

-28.94782240617039

In [26]:
Put.Rho(S, K, r, sigma, T)/100

-0.3735358847372249

In [ ]:
delta                          -0.61919
gamma                           0.00109
vega                            2.78342
theta                         -28.40972
rho                            -0.37467

### snapshot example from 30SEP2021

In [121]:
bid = 0.0095
ask = 0.0105

IV_bid = 0.854
IV_ask = 0.874
bid_price = 400.75
ask_price = 421.84
S = 42166
K = 60000
r=0
T = 30/365
sigma = IV_bid

In [122]:
print('S={},'.format(S), 'K={},'.format(K), 'T={},'.format(T), 'r={},'.format(r), 'sgima={},'.format(sigma))

S=42166, K=60000, T=0.0821917808219178, r=0, sgima=0.854,


In [115]:
call_price_IV_bid = euro_vanilla(S, K, T, r, IV_bid, option='call')
print('call price using IV bid ({}) =  {} '.format(IV_bid, call_price_IV_bid))
print('call bid price {}'.format(bid_price))

call_price_IV_ask = euro_vanilla(S, K, T, r, IV_ask, option='call')
print('call price using IV ask ({}) =  {} '.format(IV_ask, call_price_IV_ask))
print('call ask price {}'.format(ask_price))

call price using IV bid (0.854) =  410.39147470840635 
call bid price 400.75
call price using IV ask (0.874) =  451.812848334489 
call ask price 421.84


In [116]:
IV_bid = find_vol(bid_price, S, K, T, r)
IV_ask = find_vol(ask_price, S, K, T, r)

In [117]:
IV_bid, IV_ask

(0.8492054360526142, 0.8596221403404919)

In [125]:
bsm=op.black_scholes(K=K, St=S, r=r, t=T, v=sigma, type='c')

In [126]:
bsm['value']

{'option value': 0.0, 'intrinsic value': 0, 'time value': 0.0}